# pandas

> Stata-to-pandas utilities, used in `nbstata.browse`

In [ ]:
#| default_exp pandas
%load_ext autoreload
%autoreload 2

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
from nbstata.stata import stata_formatted
from nbstata.stata_more import IndexVar

## Better stata-to-pandas

Improves on [pystata.stata.pdataframe_from_data](https://www.stata.com/python/pystata/stata.html#pystata.stata.pdataframe_from_data) (and 
[pystata.stata.pdataframe_from_frame](https://www.stata.com/python/pystata/stata.html#pystata.stata.pdataframe_from_frame)) by making the pandas DataFrame index correspond to Stata observation numbers and also providing an option to output numeric values as strings using their Stata formats.

In [ ]:
#| export
def _better_dataframe(hdl, var, obs, selectvar, valuelabel, missingval):
    import pandas as pd
    with IndexVar() as idx_var:
        data = hdl.getAsDict(var, obs, selectvar, valuelabel, missingval)
        if not data:
            return pd.DataFrame()
    
        if idx_var in data:
            idx = data.pop(idx_var)
        else:
            temp_var = [idx_var, selectvar] if selectvar else idx_var
            idx = hdl.getAsDict(temp_var, obs, selectvar, valuelabel, missingval).pop(idx_var)
        idx = pd.array(idx, dtype='int64')

        return pd.DataFrame(data=data, index=idx)

#| hide

Reference Stata docs: [getAsDict example](https://blog.stata.com/2020/11/05/stata-python-integration-part-8-using-the-stata-function-interface-to-copy-data-from-stata-to-python/)

In [ ]:
#| hide
#| eval: False
from nbstata.config import launch_stata
from nbstata.stata_more import run_sfi

In [ ]:
#| hide
#| eval: False
launch_stata(splash=False)

In [ ]:
#| hide
#| eval: False
import sfi

In [ ]:
#| hide
from fastcore.test import test_eq, test_fail
import numpy as np

In [ ]:
#| hide
#| eval: False
run_sfi("""\
sysuse uslifeexp2, clear
gen sel_var = year==1920""")

(U.S. life expectancy, 1900-1940)


In [ ]:
#| hide
#| eval: False
_better_dataframe(sfi.Data, 'le', None, 'sel_var', True, np.NaN)

,le
21,54.099998


In [ ]:
#| hide
#| eval: False
_better_dataframe(sfi.Data, None, range(0, 10), 'sel_var', True, np.NaN)

""


In [ ]:
#| hide
#| eval: False
_better_dataframe(sfi.Data, None, range(-31, -1), 'sel_var', True, np.NaN)

,year,le,sel_var
21,1920,54.099998,1.0


In [ ]:
#| hide
#| eval: False
run_sfi("drop sel_var")

In [ ]:
#| export
def _simple_dataframe_from_stata(stfr, var, valuelabel, missingval):
    from pystata import stata
    if stfr is None:
        df = stata.pdataframe_from_data(var=var, valuelabel=valuelabel, missingval=missingval)
    else:
        df = stata.pdataframe_from_frame(stfr, var=var, valuelabel=valuelabel, missingval=missingval)
    df.index += 1
    return df

In [ ]:
#| hide
#| eval: False
from pystata.stata import _DefaultMissing

In [ ]:
#| hide
#| eval: False
run_sfi("""\
sysuse lifeexp, clear
keep in 1/5""")
_simple_dataframe_from_stata(None, None, False, _DefaultMissing())

NameError: name 'run_direct' is not defined

In [ ]:
#| export
def better_dataframe_from_stata(stfr, var, obs, selectvar, valuelabel, missingval, sformat):
    import numpy as np
    import pandas as pd
    import sfi
    hdl = sfi.Data if stfr is None else sfi.Frame.connect(stfr)
    custom_index_not_needed = obs is None and not selectvar
    if custom_index_not_needed:
        df = _simple_dataframe_from_stata(stfr, var, valuelabel, missingval)
    else:
        if hdl.getObsTotal() <= 0:
            return pd.DataFrame()
        df = _better_dataframe(hdl, var, obs, selectvar, valuelabel, missingval)
    if sformat:
        for v in list(df.columns):
            if hdl.isVarTypeString(v) or (valuelabel and missingval==np.NaN
                                          and pd.api.types.is_string_dtype(df[v])):
                continue
            v_format = hdl.getVarFormat(v)
            if missingval != np.NaN and pd.api.types.is_string_dtype(df[v]):
                def format_value(x):
                    return stata_formatted(x, v_format) if type(x)!=str else x
            else:
                def format_value(x):
                    return stata_formatted(x, v_format)
            df[v] = df[v].apply(format_value)
    return df

In [ ]:
#| export
def better_pdataframe_from_data(var=None, obs=None, selectvar=None, valuelabel=False, missingval=None, sformat=False):
    import numpy as np
    if missingval is None:
        missingval = np.NaN
    return better_dataframe_from_stata(None, var, obs, selectvar, valuelabel, missingval, sformat)

In [ ]:
#| export
def better_pdataframe_from_frame(stfr, var=None, obs=None, selectvar=None, valuelabel=False, missingval=None, sformat=False):
    import numpy as np
    if missingval is None:
        missingval = np.NaN
    return better_dataframe_from_stata(stfr, var, obs, selectvar, valuelabel, missingval, sformat)

In [ ]:
#| eval: False
run_sfi("""\
replace le = . if _n==5
replace year = 2022 if year==1900
format year %-ty
gen str_var = "test string" if _n!=5
gen date = dofy(year)
format date %td
gen double date_tc = cofd(date)
format date_tc %tc
gen double date_tC = Cofd(date)
format date_tC %tC""")
better_pdataframe_from_data().head()

In [ ]:
#| eval: False
better_pdataframe_from_data(sformat=True).head()

In [ ]:
#| hide
#| eval: False
better_pdataframe_from_data(valuelabel=True, sformat=True, missingval=".").head()

In [ ]:
#| hide
#| eval: False
better_pdataframe_from_data(sformat=True, missingval=".").head()

In [ ]:
#| hide
#| eval: False
better_pdataframe_from_data(obs=range(1, 3)).head()

In [ ]:
#| hide
#| eval: False
test_fail(better_pdataframe_from_data, kwargs=dict(obs=range(1000)))

In [ ]:
#| hide
#| eval: False
better_pdataframe_from_data(missingval='.').head()

In [ ]:
#| hide
#| eval: False
better_pdataframe_from_data(var=['year', 'date_tc', 'date_tC']).head()

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()